### Deploy and Interact with Consent based Smart Contract
The aim of this notebook is to compile, deploy and interact with consent based smart contract

# Compile contract from file

In [14]:
# Import libraries
import json
import web3
from datetime import datetime
from web3 import Web3

# import py_solc_x as px
import solcx
from web3.contract import Contract
import os

# from brownie import  web3

# import icd10

In [15]:
# Read in LUCE contract code
# solcx.install_solc('0.8.0')
print(os.getcwd())
consent_fp = r"data\UnifiedConsentModel.sol"
# consent_fp_relative = r"jupyter\\data\\UnifiedConsentModel.sol"
with open(consent_fp) as file:
    contract_source_code = file.read()

# Compile & Store Compiled source code
# print(contract_source_code)
solcx_version = "0.8.17"
# solcx_version = '0.5.16'
# solcx.install_solc(solcx_version)
# compiled_sol = solcx.compile_source(source=contract_source_code,
#   solc_binary="/snap/bin/solc")
compiled_sol = solcx.compile_source(
    source=contract_source_code,
    # solc_binary='/usr/bin/solc',
    # optimize_runs=200,
    output_values=["abi", "bin", "bin-runtime"],
    optimize=True,
    solc_version=solcx_version,
)

# compiled_sol = solcx.compile_files(source_files=consent_fp,
#                                     # solc_binary='/usr/bin/solc',
#                                     # optimize_runs=200,
#                                     output_values=["abi", "bin",'bin-runtime'],
#                                     # optimize=True,
#                                     solc_version = solcx_version
#                                     )

print(compiled_sol.keys())
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol["<stdin>:ConsentCode"]
# contract_interface = compiled_sol['data/ConsentContractDate.sol:ConsentCode']
print(contract_interface.keys())
# contract_interface
# Extract abi and bytecode
abi = contract_interface["abi"]
bytecode = contract_interface["bin"]
### Deploy
startTime = datetime.now()
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

# compiled_contract = web3.eth.compile_contract(contract_source)
# abi = compiled_contract["abi"]
# bytecode = compiled_contract["bytecode"]
# contract_address = web3.eth.contract(abi=abi, bytecode=bytecode).deploy()

# assert True is w3.isConnected()
# Use local Ganache GUI on macOS
# w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]
w3.eth.accounts
# The default `eth.defaultAccount` address is used as the default "from" property for request_1_address dictionaries if no other explicit "from" property is specified.
# Create contract blueprint
contract = w3.eth.contract(abi=abi, bytecode=bytecode)
# Submit the request_1_address that deploys the contract
provider_address = w3.eth.accounts[0]
provider_address_sum = Web3.to_checksum_address(provider_address)

request_1_address = w3.eth.accounts[1]
request_1_address_sum = Web3.to_checksum_address(request_1_address)

request_2_address = w3.eth.accounts[2]
request_2_address_sum = Web3.to_checksum_address(request_1_address)

tx_hash = contract.constructor().transact({"from": provider_address, "gas": int(1e7)})


tx_hash
### Obtain Transcation Receipt
# Wait for the request_1_address to be mined, and get the request_1_address receipt
# tx_receipt = w3.eth.waitForrequest_1_addressReceipt(tx_hash)

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=120, poll_latency=0.1)

# We obtain the block number under which it is deployed
# global contract_block
contract_block = w3.eth.block_number
print("The contract is deployed with block number", contract_block, ".")
# With obtain the final address of the contract

# global contract_address
contract_address = tx_receipt.contractAddress

print("The contract has the address", contract_address)
gas_limit = int(1e7)
provider_dict = {
    "from": provider_address,
    "to": contract_address,
    "value": 0,
    "gas": gas_limit,
    # 'gasPrice': w3.eth.gas_price*0.1,
    # 'nonce': 33 ,
}

requester_1_dict = {
    "from": request_1_address,
    "to": contract_address,
    "value": 0,
    "gas": gas_limit,
    # 'gasPrice': w3.eth.gas_price*0.1,
    # 'nonce': 33 ,
}

requester_2_dict = {
    "from": request_2_address,
    "to": contract_address,
    "value": 0,
    "gas": gas_limit,
    # 'gasPrice': w3.eth.gas_price*0.1,
    # 'nonce': 33 ,
}
### Interact with contract
# Create python instance of deployed contract
# caddress = '0x3FEAfC9084e95BC5B07FBbBd197Af22422A46019'
contract = w3.eth.contract(
    address=contract_address,
    abi=abi,
    # bytecode=bytecode,
)
# Extract default accounts created by ganache
accounts = w3.eth.accounts
accounts

d:\WorkSpace\Maastricht\LUCE\DecentralizedHealthcareBackend\jupyter
dict_keys(['<stdin>:ConsentCode'])
dict_keys(['abi', 'bin', 'bin-runtime'])
The contract is deployed with block number 2648 .
The contract has the address 0x3b528DE78C85Bf490DECCD8546dCa4cB2db2Bc1B


['0x84dF8A457982333C0893b968872F57b963243307',
 '0x159f33a7b5a9fB939b8AC6f8CE2E54e7eaE13bE0',
 '0x68C0eC5c56f0EDe43affA748e5D4aFd87f31513d']

# Test cases

## Disease Name

In [16]:
import re

pattern = r"^[A-Z\*][0-9\*]{2}\.[0-9\*]$"
pattern_compiled = re.compile(pattern)




def diseaseCode2Int(code:str) -> int:
    # r = fx.UInt16(0)
    # print(bin(r))
    # since A00.0 is a correct code, so plus 1 for each level
    if not pattern_compiled.match( code):
        print(f"The string {code} not matches the pattern")
        return 0
    chapter_str = code[0]
    group_str = code[1:3]
    category_str = code[-1]
    result = 0
    if chapter_str == '*':
        return 0
        # chapter_int = 0
    else:
        chapter_int = int(ord(code[0]) - ord('A'))+1
        b_chapter = (chapter_int) << 11

    if group_str == '**':
        return b_chapter
    else:
        group_int = int(code[1:3])+1
        b_group = (group_int) << 4
    
    if category_str == '*':
        return b_chapter + b_group
    else:
        category_int = int(code[-1])+1
        return  b_chapter + b_group + category_int


aCode = "A00.1"
print(bin(diseaseCode2Int(aCode)))

aCode = "*00.1"
print(bin(diseaseCode2Int(aCode)))

aCode = "A**.1"
print(bin(diseaseCode2Int(aCode)))
aCode = "A00.*"
print(bin(diseaseCode2Int(aCode)))

aCode = "000.*"
print(bin(diseaseCode2Int(aCode)))

0b100000010010
0b0
0b100000000000
0b100000010000
The string 000.* not matches the pattern
0b0


In [19]:
print(contract.address)
print(contract.functions)


# print(contract.checkAccess(requester_1_dict))


class Person:
    address = 0
    name = ""
    description = ""

    boolItems = BooleanItems()

    DiseaseItems = []
    Country_Code = []
    Group_Code = 0
    Country_Group_Code = 0
    Start_Year = 0
    Start_Month = 0
    Start_Day = 0
    Months = 0

    Contract = None
    person_dict = None
    role=0


class Provider(Person):
    def __init__(self):
        self.role = 0
    
    def uploadSimpleItems(self):
        self.Contract.functions.UploadSimpleItems(self.role,
            self.address, self.boolItems.to_int()
        ).transact(self.person_dict)

    def UploadDiseaseItems(self):
        self.Contract.functions.UploadDiseaseItemsProvider(
            self.address, self.DiseaseItems
        ).transact(self.person_dict)

    def UploadCountryItems(self):
        self.Contract.functions.UploadCountryItemsProvider(
            self.address, self.CountryItems
        ).transact(self.person_dict)

    def UploadDate(self):
        self.Contract.functions.UploadDateProvider(
            self.address, self.Start_Year, self.Start_Month, self.Start_Day, self.Months
        ).transact(self.person_dict)

    def __str__(self):
        return f"Provider: {self.name}, {self.address}, {self.description}"
    
    def display(self):
        print(self.__str__())


class BooleanItems:
    ClinicalProfessionals = False
    AcademicProfessionals = False
    ReferenceOrControlMaterial = False
    MethodsDevelopment = False
    PopulationsResearch = False
    AncestryResearch = False
    FundamentalBioResearch = False
    DrugDevelopmentResearch = False
    AgeCategoriesResearch = False
    GenderCategoriesResearch = False
    ProfitPurpose = False
    ProfitMakingProfessionals = False
    FormalApprovalRequired = False

    def to_list(self):
        return [
            self.ClinicalProfessionals,
            self.AcademicProfessionals,
            self.ReferenceOrControlMaterial,
            self.MethodsDevelopment,
            self.PopulationsResearch,
            self.AncestryResearch,
            self.FundamentalBioResearch,
            self.DrugDevelopmentResearch,
            self.AgeCategoriesResearch,
            self.GenderCategoriesResearch,
            self.ProfitPurpose,
            self.ProfitMakingProfessionals,
            self.FormalApprovalRequired,
        ]

    def to_int(self):
        result = 0
        bool_list = self.to_list()
        for i in range(len(bool_list)):
            result += bool_list[i] << i
        return result


provider1 = Provider()
provider1.address = provider_address
provider1.name = "Provider1"
provider1.description = "Provider1"
provider1.Contract = contract
provider1.person_dict = provider_dict

provider1.boolItems.ClinicalProfessionals = True
provider1.uploadSimpleItems()

bItems = BooleanItems()
bItems.ClinicalProfessionals = True
bItems.AncestryResearch = True
bItems.ProfitPurpose = True

print(bItems.to_int())
result = contract.functions.UploadSimpleItemsProvider(
    provider_address, bItems.to_int()
).transact(provider_dict)

print("2 is success")

# result = contract.functions.DisplayProviderBooleanItems(provider_address).call()
# result = contract.functions.AccessData1().call()

# print(result)
# print(contract.functions._functions)

UploadProviderBooleanItems = contract.functions.UploadSimpleItemsProvider(
    provider_address, bItems.to_int()
)
gas_estimate = UploadProviderBooleanItems.estimate_gas(
    {
        "from": provider_address,
        # "gas": 200000,  # You can set an initial gas limit here
    }
)
print(f"Estimated gas cost for UploadProviderBooleanItems: {gas_estimate}")
AccessData = contract.functions.AccessData(provider_address, request_1_address)
gas_estimate = AccessData.estimate_gas(
    {
        "from": provider_address,
        # "gas": 200000,  # You can set an initial gas limit here
    }
)
print(f"Estimated gas cost for AccessData: {gas_estimate}")
result = AccessData.call()
print(result)

0x3b528DE78C85Bf490DECCD8546dCa4cB2db2Bc1B
1057
2 is success
Estimated gas cost for UploadProviderBooleanItems: 28688
Estimated gas cost for AccessData: 38275
0


In [ ]:
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, False, False, False, False, False, False, False).transact(provider_dict)
tx_hash = contract.functions.UploadProviderDate(provider_address, 2023, 6, 27, 6).transact(provider_dict)
tx_hash = contract.functions.giveResearchPurpose(request_1_address, True, True, False, False, False).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, True, False, True, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, False, True).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterDate(request_1_address, 2023,6,29,2023,8,17).transact(requester_1_dict)

## ***positive test***
provider_disease_code_list = ["A00.1", "C00.*","B**.*","*92.0"]
requester_disease_code_list = ["A00.1", "C00.1","B00.3", "A00.4"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]

result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.AccessData(provider_address, request_1_address).call()
result = contract.functions.AccessData(provider_address, request_1_address).call()
print(result)
assert result == 0



ABIFunctionNotFound: ("The function 'displayProviderDiseasePurpose' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')

In [ ]:
provider_disease_code_list = ["A00.1", "C00.*","B**.*"]
requester_disease_code_list = ["B00.3"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]
tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_2_address,requester_disease_code_list).transact(requester_2_dict)

result = contract.functions.AccessDisease(provider_address, request_2_address).call()
print(result)
assert result == 0

# ***negitive test***
# _OpenToGeneralResearchAndClinicalCare is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, True, True, True).transact(provider_dict)
provider_disease_code_list = ["A00.1", "C00.*","B**.*"]
requester_disease_code_list = ["D00.3", "A00.4"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result != 0
# _OpenToHMBResearch is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, False, True, True).transact(provider_dict)
provider_disease_code_list = ["A00.1", "C00.*","B**.*"]
requester_disease_code_list = ["C00.3", "A00.4"]
provider_disease_code_list = [diseaseCode2Int(code) for code in provider_disease_code_list]
requester_disease_code_list = [diseaseCode2Int(code) for code in requester_disease_code_list]

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result != 0

# _OpenToDiseaseSpecific is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, False).transact(provider_dict)
tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)
result = contract.functions.displayProviderPrimaryCategorye(provider_address).call()
print(result)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result != 0

# code not equal is false
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, provider_disease_code_list ).transact(provider_dict)
tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,requester_disease_code_list).transact(requester_1_dict)
result = contract.functions.displayProviderDiseasePurpose(provider_address).call()
print(result)
result = contract.functions.displayRequesterDiseasePurpose(request_1_address).call()
print(result)
result = contract.functions.displayProviderPrimaryCategorye(provider_address).call()
print(result)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert result == 0

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, False, False, False, False, False, False, False).transact(provider_dict)
# tx_hash = contract.functions.UploadProviderDate(provider_address, 2023, 6, 27, 6).transact(provider_dict)

tx_hash = contract.functions.UploadProviderDiseasePurpose(provider_address, requester_disease_code_list ).transact(provider_dict)

tx_hash = contract.functions.giveResearchPurpose(request_1_address, True, True, False, False, False).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, True, False, True, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, False, True).transact(requester_1_dict)
# tx_hash = contract.functions.GiveRequesterDate(request_1_address, 2023,6,29,2023,8,17).transact(requester_1_dict)

tx_hash = contract.functions.GiveRequesterDiseasePurpose(request_1_address,provider_disease_code_list).transact(requester_1_dict)
result:str= contract.functions.AccessDisease(provider_address, request_1_address).call()
print(result)
assert  result != 0


ABIFunctionNotFound: ("The function 'UploadProviderDiseasePurpose' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')

## Country


In [ ]:
import countrygroups as cg
import pycountry as pc
import os 
import csv
import json

country_fp = 'data/country-codes.csv'
country_line_fp = 'data/country-codes.jsonl'
country_code_rows = list(csv.DictReader(open(country_fp, 'r')))
# ['official_name_en']

print(cg.EUROPEAN_UNION)
country_code_dict = {row['ISO3166-1-Alpha-3']: row for row in country_code_rows}
country_index_dict = {row['ISO3166-1-Alpha-3']: i for i, row in enumerate(country_code_rows)}
group_name =['ANNEX_ONE', 'ANNEX_ONE_KAZ', 'AOSIS', 'AR5', 'ARAB_GROUP', 'BRICS', 'EIG', 'EUROPEAN_UNION', 'G20', 'G7', 'G77', 'GRADUATED_LDCS', 'IMO', 'LDC', 'LLDC', 'MONTREAL_PROTOCOL_HIGH_AMBIENT_TEMP', 'NON_ANNEX_ONE', 'OECD', 'OPEC', 'SIDS', 'SIDS_NON_UN_OR_REGIONAL_COMMISSIONS_ASSOCIATES', 'SSP', 'UMBRELLA', 'UN_REGIONAL_GROUPS', 'UNFCCC', 'UNSTATS_GEOGRAPHICAL_REGIONS', 'MONTREAL_PROTOCOL_COUNTRIES']
gn_key = 'groups'
group_index_dict = {gn: 2**i for i, gn in enumerate(group_name)}

for gn in group_name:
    countries_gn =  cg.__dict__.get(gn)
    for country in countries_gn:
        gn_dict = country_code_dict.get(country)
        if gn_dict is None:
            print(gn)
            print(country)
            continue
        if gn_key not in gn_dict:
            gn_dict[gn_key] = list()
        gn_dict[gn_key].append(gn)
  
with open(country_line_fp, 'w') as f:
    text = ''
    for row in country_code_rows:
        text += json.dumps(row) + '\n'
    f.write(text)
# print(country_code_rows[1])
# print(cg.__dict__.
    #   keys())
# print(os.getcwd())

# print(pc.countries)

def index_for_requester(country_names:str = None, group_names:str = None):
    group_index = 0
    country_index = 0
    if group_names is not None:
        for gn in group_names:
            group_index += group_index_dict[gn]
    if country_names is not None:
        for cn in country_names:
            country_index += country_index_dict[cn]
            group_name_cn = country_code_dict[cn].get(gn_key)
            for gn in group_name_cn:
                group_index += group_index_dict[gn]
    return group_index,country_index

def index_for_provider(country_names:str = None, group_names:str = None):
    group_index = 0
    country_index = 0
    if group_names is not None:
        for gn in group_names:
            group_index += group_index_dict[gn]
    if country_names is not None:
        for cn in country_names:
            country_index += country_index_dict[cn]
    return  group_index,country_index

# print(index_for_requester(group_names=['EUROPEAN_UNION']))
# print(index_for_requester(country_names=['CHN']))
# print(index_for_requester(country_names=['CHN'], group_names=['EUROPEAN_UNION']))
# print(index_for_requester(country_names=['CHN'], group_names=['EUROPEAN_UNION', 'G7']))
# print(index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7']))
# print(index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD']))

# print(index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20']))
# print(index_for_provider(country_names=['CHN'], group_names=['EUROPEAN_UNION']))
# print(index_for_provider(country_names=['CHN'], group_names=['EUROPEAN_UNION', 'G7']))
# print(index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION']))

FileNotFoundError: [Errno 2] No such file or directory: 'data/country-codes.csv'

In [ ]:
# GeographicSpecificRestriction is true, UseBySpecifiedCountries is True, Country meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, True, False, False, False, False,False,False,False).transact(provider_dict)
provider_group_index, provider_country_index = index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
tx_hash = contract.functions.UploadProviderCountry(provider_address, country_index, group_index).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,True).transact(requester_1_dict)
requester_group_index,requester_country_index = index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,country_index, group_index).transact(requester_1_dict)
OpenToNonProfitUseOnly = contract.functions.displayProviderDataRequirement(provider_address).call(provider_dict)
# tx_hash = Contract.functions.giveProfit(request_1_address,2).transact(requester_1_dict)
print("openNoProfit",OpenToNonProfitUseOnly)
requester_code = contract.functions.displayRequesterCountry( request_1_address).call()
provider_code = contract.functions.displayProviderCountry( provider_address).call()
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(f"requester_code is {requester_code}, provider_code is {provider_code} ")
print(result)
assert  result == 0

# GeographicSpecificRestriction is False ,  UseBySpecifiedCountries is True, Country not  meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, False, True, True, True,True,True,True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderCountry(provider_address, provider_group_index, provider_country_index).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,True).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,
                                                  requester_group_index,requester_country_index
                                                  ).transact(requester_1_dict)
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(result)
assert  result == 0

provider_group_index, provider_country_index = index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
requester_group_index,requester_country_index = index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
# GeographicSpecificRestriction is True ,  UseBySpecifiedCountries is False, Country not  meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, True, True, True, True, True,True,True,True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderCountry(provider_address, 
                                                   provider_group_index, provider_country_index
                                                   ).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,True).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,requester_group_index,requester_country_index
                                                  ).transact(requester_1_dict)
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(result)
assert  result == 0

provider_group_index, provider_country_index = index_for_provider(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
requester_group_index,requester_country_index = index_for_requester(country_names=['CHN', 'USA'], group_names=['EUROPEAN_UNION', 'G7', 'OECD', 'G20'])
# GeographicSpecificRestriction is True ,  UseBySpecifiedCountries is False, Country   meet
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, True, True, True, True,True,True,True).transact(provider_dict)
tx_hash = contract.functions.UploadProviderCountry(provider_address, 
                                                   provider_group_index, provider_country_index 
                                                   ).transact(provider_dict)
tx_hash = contract.functions.giveGeographicSpecificRestriction(request_1_address,False).transact(requester_1_dict)
tx_hash = contract.functions.GiveRequesterCountry(request_1_address,
                                                  requester_group_index,requester_country_index
                                                  ).transact(requester_1_dict)
result:str= contract.functions.AccessCountry(provider_address, request_1_address).call()
print(result)
assert  result == 0



: 

## Others

In [ ]:
startTime14 = datetime.now()
#Data Requesters Account
# request_1_address = accounts[0]
#Data Requester 2 (HMB and Profit Making)
tx_hash = contract.functions.giveHMBPurpose(request_1_address, True, True, True, False, True, True).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, False, True, True, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, True, False).transact(requester_1_dict)
endTimeDataRequester14 = datetime.now()
elapsedTimeDataRequester14 = endTimeDataRequester14 - startTime14
print("elapsedTimeDataRequester14", elapsedTimeDataRequester14)
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData14 = datetime.now()
#elapsedTimeDataRequesterAccessData14 = endTimeDataRequesterAccessData14 - startTime14
#print("elapsedTimeDataRequesterAccessData14", elapsedTimeDataRequesterAccessData14)
startTime15 = datetime.now()
#Data Requesters Account  
request_1_address = accounts[0]
#Data Requester 3 (Disease Specific and Non-Profit-Making)
tx_hash = contract.functions.giveHMBPurpose(request_1_address, False, False, False, True, False, False).transact(requester_1_dict)
tx_hash = contract.functions.givePerson(request_1_address, False, True, False, False).transact(requester_1_dict)
tx_hash = contract.functions.giveProfit(request_1_address, False, True).transact(requester_1_dict)
endTimeDataRequester15 = datetime.now()
elapsedTimeDataRequester15 = endTimeDataRequester15 - startTime15
print("elapsedTimeDataRequester15", elapsedTimeDataRequester15)
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData15 = datetime.now()
elapsedTimeDataRequesterAccessData15 = endTimeDataRequesterAccessData15 - startTime15
print("elapsedTimeDataRequesterAccessData15", elapsedTimeDataRequesterAccessData15)
startTime1 = datetime.now()
#Data provider consent (General research & profit making)
#D-002
# test_address1 = accounts[1]
tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, True, True, True).transact(provider_dict)
endTimeDataProvider1 = datetime.now()
elapsedTimeDataProvider1 = endTimeDataProvider1 - startTime1
print("elapsedTimeDataProvider1", elapsedTimeDataProvider1)
startTime1_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData1_13 = datetime.now()
elapsedTimeDataRequesterAccessData1_13 = endTimeDataRequesterAccessData1_13 - startTime1_13
print("elapsedTimeDataRequesterAccessData1_13", elapsedTimeDataRequesterAccessData1_13)
startTime1_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData1_14 = datetime.now()
elapsedTimeDataRequesterAccessData1_14 = endTimeDataRequesterAccessData1_14 - startTime1_14
print("elapsedTimeDataRequesterAccessData1_14", elapsedTimeDataRequesterAccessData1_14)
startTime1_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData1_15 = datetime.now()
elapsedTimeDataRequesterAccessData1_15 = endTimeDataRequesterAccessData1_15 - startTime1_15
print("elapsedTimeDataRequesterAccessData1_15", elapsedTimeDataRequesterAccessData1_15)
#startTime2 = datetime.now()
#Data provider consent (General research & non-profit making)
#H-001 
# test_address2 = accounts[2]

tx_hash = contract.functions.UploadDataPrimaryCategory(request_1_address, False, True, False, False, False).transact(requester_1_dict)
tx_hash = contract.functions.UploadDataRequirements(request_1_address, False, True, False, False, False, False, False, False).transact(requester_1_dict)
#endTimeDataProvider2 = datetime.now()
#elapsedTimeDataProvider2 = endTimeDataProvider2 - startTime2
#print("elapsedTimeDataProvider2", elapsedTimeDataProvider2)
#startTime2_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData2_13 = datetime.now()
#elapsedTimeDataRequesterAccessData2_13 = endTimeDataRequesterAccessData2_13 - startTime2_13
#print("elapsedTimeDataRequesterAccessData2_13", elapsedTimeDataRequesterAccessData2_13)
#startTime2_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData2_14 = datetime.now()
#elapsedTimeDataRequesterAccessData2_14 = endTimeDataRequesterAccessData2_14 - startTime2_14
#print("elapsedTimeDataRequesterAccessData2_14", elapsedTimeDataRequesterAccessData2_14)
#startTime2_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData2_15 = datetime.now()
#elapsedTimeDataRequesterAccessData2_15 = endTimeDataRequesterAccessData2_15 - startTime2_15
#print("elapsedTimeDataRequesterAccessData2_15", elapsedTimeDataRequesterAccessData2_15)#
#startTime3 = datetime.now()
#Data provider consent (General research & profit making)
#H-002 
test_address3 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, True, False, False, False).transact(provider_dict)
#endTimeDataProvider3 = datetime.now()
#elapsedTimeDataProvider3 = endTimeDataProvider3 - startTime3
#print("elapsedTimeDataProvider3", elapsedTimeDataProvider3)
#startTime3_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData3_13 = datetime.now()
#elapsedTimeDataRequesterAccessData3_13 = endTimeDataRequesterAccessData3_13 - startTime3_13
#print("elapsedTimeDataRequesterAccessData3_13", elapsedTimeDataRequesterAccessData3_13)
#startTime3_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData3_14 = datetime.now()
#elapsedTimeDataRequesterAccessData3_14 = endTimeDataRequesterAccessData3_14 - startTime3_14
#print("elapsedTimeDataRequesterAccessData3_14", elapsedTimeDataRequesterAccessData3_14)
#startTime3_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
#endTimeDataRequesterAccessData3_15 = datetime.now()
#elapsedTimeDataRequesterAccessData3_15 = endTimeDataRequesterAccessData3_15 - startTime3_15
#print("elapsedTimeDataRequesterAccessData3_15", elapsedTimeDataRequesterAccessData3_15)
#startTime4 = datetime.now()
#Data provider consent (HMB research & non-profit making)
#D-003 
test_address4 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address4, False, False, True, False, True).transact(requester_1_dict)
tx_hash = contract.functions.UploadDataRequirements(test_address4, False, True, False, False, False, False, False, False).transact(requester_1_dict)

#endTimeDataProvider4 = datetime.now()
#elapsedTimeDataProvider4 = endTimeDataProvider4 - startTime4
#print("elapsedTimeDataProvider4", elapsedTimeDataProvider4)
#startTime4_13 = datetime.now()
contract.functions.AccessData(test_address4, request_1_address).call()
#endTimeDataRequesterAccessData4_13 = datetime.now()
#elapsedTimeDataRequesterAccessData4_13 = endTimeDataRequesterAccessData4_13 - startTime4_13
#print("elapsedTimeDataRequesterAccessData4_13", elapsedTimeDataRequesterAccessData4_13)
#startTime4_14 = datetime.now()
contract.functions.AccessData(test_address4, request_1_address).call()
#endTimeDataRequesterAccessData4_14 = datetime.now()
#elapsedTimeDataRequesterAccessData4_14 = endTimeDataRequesterAccessData4_14 - startTime4_14
#print("elapsedTimeDataRequesterAccessData4_14", elapsedTimeDataRequesterAccessData4_14)
#startTime4_15 = datetime.now()
contract.functions.AccessData(test_address4, request_1_address).call()
#endTimeDataRequesterAccessData4_15 = datetime.now()
#elapsedTimeDataRequesterAccessData4_15 = endTimeDataRequesterAccessData4_15 - startTime4_15
#print("elapsedTimeDataRequesterAccessData4_15", elapsedTimeDataRequesterAccessData4_15)
startTime5 = datetime.now()
#Data provider consent (HMB research & profit making)
#D-004 
test_address5 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address5, False, False, True, False, True).transact(requester_1_dict)
endTimeDataProvider5 = datetime.now()
elapsedTimeDataProvider5 = endTimeDataProvider5 - startTime5
print("elapsedTimeDataProvider5", elapsedTimeDataProvider5)
startTime5_13 = datetime.now()
contract.functions.AccessData(test_address5, request_1_address).call()
endTimeDataRequesterAccessData5_13 = datetime.now()
elapsedTimeDataRequesterAccessData5_13 = endTimeDataRequesterAccessData5_13 - startTime5_13
print("elapsedTimeDataRequesterAccessData5_13", elapsedTimeDataRequesterAccessData5_13)
startTime5_14 = datetime.now()
contract.functions.AccessData(test_address5, request_1_address).call()
endTimeDataRequesterAccessData5_14 = datetime.now()
elapsedTimeDataRequesterAccessData5_14 = endTimeDataRequesterAccessData5_14 - startTime5_14
print("elapsedTimeDataRequesterAccessData5_14", elapsedTimeDataRequesterAccessData5_14)
startTime5_15 = datetime.now()
contract.functions.AccessData(test_address5, request_1_address).call()
endTimeDataRequesterAccessData5_15 = datetime.now()
elapsedTimeDataRequesterAccessData5_15 = endTimeDataRequesterAccessData5_15 - startTime5_15
print("elapsedTimeDataRequesterAccessData5_15", elapsedTimeDataRequesterAccessData5_15)
#Data provider consent (HMB research & non-profit making)
#H-003 
test_address6 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address6, False, False, True, False, False).transact(requester_1_dict)
tx_hash = contract.functions.UploadDataRequirements(test_address6, False, True, False, False, False, False, False, False).transact(requester_1_dict)
contract.functions.AccessData(test_address6, request_1_address).call()
contract.functions.AccessData(test_address6, request_1_address).call()
contract.functions.AccessData(test_address6, request_1_address).call()
#Data provider consent (HMB research & profit making)
#H-004 
test_address7 = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(test_address7, False, False, True, False, False).transact(requester_1_dict)
contract.functions.AccessData(test_address7, request_1_address).call()
contract.functions.AccessData(test_address7, request_1_address).call()
contract.functions.AccessData(test_address7, request_1_address).call()
startTime8 = datetime.now()
#Data provider consent (Disease-specific research & non-profit making)
#D-005 

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, True).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, True, False, False, False, False, False, False).transact(provider_dict)
endTimeDataProvider8 = datetime.now()
elapsedTimeDataProvider8 = endTimeDataProvider8 - startTime8
print("elapsedTimeDataProvider8", elapsedTimeDataProvider8)
#Specify start and end dates

tx_hash = contract.functions.UploadProviderDate(provider_address, 2023,6,11,3).transact(provider_dict)
endTimeDataProvider8 = datetime.now()
elapsedTimeDataProvider8 = endTimeDataProvider8 - startTime8
print("elapsedTimeDataProvider8", elapsedTimeDataProvider8)
startTime8_13 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData8_13 = datetime.now()
elapsedTimeDataRequesterAccessData8_13 = endTimeDataRequesterAccessData8_13 - startTime8_13
print("elapsedTimeDataRequesterAccessData8_13", elapsedTimeDataRequesterAccessData8_13)
startTime8_14 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData8_14 = datetime.now()
elapsedTimeDataRequesterAccessData8_14 = endTimeDataRequesterAccessData8_14 - startTime8_14
print("elapsedTimeDataRequesterAccessData8_14", elapsedTimeDataRequesterAccessData8_14)
startTime8_15 = datetime.now()
contract.functions.AccessData(provider_address, request_1_address).call()
endTimeDataRequesterAccessData8_15 = datetime.now()
elapsedTimeDataRequesterAccessData8_15 = endTimeDataRequesterAccessData8_15 - startTime8_15
print("elapsedTimeDataRequesterAccessData8_15", elapsedTimeDataRequesterAccessData8_15)
#Data provider consent (Disease-specific research & profit making)
# D-006 

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, True).transact(provider_dict)
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
#Data provider consent (Disease-specific research & non-profit making)
#H-005 
provider_address = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, False).transact(provider_dict)
tx_hash = contract.functions.UploadDataRequirements(provider_address, False, True, False, False, False, False, False, False).transact(provider_dict)
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
#Data provider consent (Disease-specific research & profit making)
#H-006 
provider_address = accounts[0]

tx_hash = contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, False).transact(provider_dict)
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
contract.functions.AccessData(provider_address, request_1_address).call()
endTime1 = datetime.now()
elapsedTime1 = endTime1 - startTime
print("elapsedTime1", elapsedTime1)
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
#print("Gas", Contract.functions.UploadDataPrimaryCategory(provider_address, False, False, False, False, True).estimateGas())
#print("Gas", Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).estimateGas())
#print("Gas", Contract.functions.giveResearchPurpose(request_1_address, True, True, False, False, False).estimateGas())
#print("Gas", Contract.functions.givePerson(request_1_address, True, False, True, False).estimateGas())
#print("Gas", Contract.functions.giveProfit(request_1_address, True, False).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, request_1_address).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, request_1_address).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, request_1_address).estimateGas())

: 